In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
import pandas as pd
import os
a=pd.read_csv('../input/stanford-car-dataset-by-classes-folder/names.csv')
car_names=a['AM General Hummer SUV 2000'].tolist() #csv file values to list
print(car_names)
len(car_names)

In [ ]:
car_names.index('Ram C/V Cargo Van Minivan 2012')
car_names[172]
car_names = ['Ram C-V Cargo Van Minivan 2012' if  i == 'Ram C/V Cargo Van Minivan 2012' else i for i in car_names] 
car_names[172]

In [ ]:
#training dataset not balanced beacause different number of image in different categories.
#so some changes eeded for better prediction
DATADIR = '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train'
number_of_images_in_each_car_folder=[]
for i in car_names:
    number_of_images_in_each_car_folder.append(len(os.listdir(os.path.join(DATADIR,i))))
print(number_of_images_in_each_car_folder)    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm



for car_name in car_names:
    path = os.path.join(DATADIR,car_name)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array , cmap='gray')
        plt.show()
        break
    break

In [ ]:
print(img_array)
img_array.shape

In [ ]:
#checking image quality with resized values
img_len=150
img_wid=100
new_array = cv2.resize(img_array, (img_len,img_wid))
plt.imshow(new_array, cmap='gray')
plt.show()

In [ ]:
new_array.shape

In [ ]:
#prepare_training_data()
training_data = []
def create_training_data():
    
    for car_name in car_names:
        path = os.path.join(DATADIR,car_name)
        class_num = car_names.index(car_name)#giving index number to each car names in list
        
        for img in tqdm(os.listdir(path)): #iterate over each image , folder names each car's names
            img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) #converting to array
            new_array = cv2.resize(img_array,(img_len,img_wid))
            training_data.append([new_array,class_num])#creating training list which include car index and image array
        

In [ ]:

create_training_data()
print("total number of image in training list",len(training_data))

In [ ]:
import random
random.shuffle(training_data)
for sample in training_data[:10]:      #to check shuffled or not
    print(sample[1])
print(training_data[0])
    


In [ ]:
x = []
y = []

for features,label in training_data:
    x.append(features)
    y.append(label)

#print(x[0].reshape(-1, img_len, img_wid, 1))

y = np.array(y)
x = np.array(x).reshape(-1, img_len, img_wid, 1) #reshaping the array as suitable to feed the cnvolutional network
print(x[0])
print("________________")
print(y[10])

In [ ]:
print(x.shape,y.shape)

In [ ]:
import pickle
pickle_out = open("x.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

#pickle_in = open("x.pickle","rb")
#x = pickle.load(pickle_in)

#pickle_in = open("y.pickle","rb")
#y = pickle.load(pickle_in) 

In [ ]:
#plt.imshow(x[0],cmap='gray')
#plt.show()



print(x.shape,y.shape)
y[:10]


In [ ]:
#preparing test data
TEST_DIR= '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test'
testing_data = []
def create_testing_data():
    
    for car_name in car_names:
        path = os.path.join(TEST_DIR,car_name)
        class_num = car_names.index(car_name)#giving index number to each car names in list
        
        for img in tqdm(os.listdir(path)): #iterate over each image is folder names each car's names
            img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) #converting to array
            new_array = cv2.resize(img_array,(img_len,img_wid))
            testing_data.append([new_array,class_num])#creating testing list which include car index and image array
    

create_testing_data()
print("total number of image in testing list",len(testing_data))

In [ ]:
    random.shuffle(testing_data)
    x_test = []
    y_test = []

    for features,label in testing_data:
        x_test.append(features)
        y_test.append(label)

    #print(x[0].reshape(-1, img_len, img_wid, 1))

    y_test = np.array(y)
    x_test = np.array(x).reshape(-1, img_len, img_wid, 1) #reshaping the array as suitable to feed the cnvolutional network


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models 
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten
from tensorflow.keras.layers import Conv2D , MaxPooling2D
from tensorflow.keras import layers

pickle_in = open("x.pickle","rb")
x = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in) 

x = x/255.0

cnn = models.Sequential([
    layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu',input_shape=(x.shape[1:])),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(195,activation='softmax')
])


In [ ]:
cnn.summary()

In [ ]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


cnn.fit(x, y, epochs=10)

In [ ]:
cnn.evaluate(x_test,y_test)

In [ ]:
test_image_path = '../input/testing-images/dda6cc52cf3c2bb22598e1c39497daee.jpg'
img_test_array = cv2.imread(test_image_path,cv2.IMREAD_GRAYSCALE)
#plt.imshow(img_test_array,cmap = 'gray')
#plt.show()
n_img_test_array=cv2.resize(img_test_array,(img_len,img_wid))
#plt.imshow(n_img_test_array, cmap='gray')
#plt.show()
test_pctr = np.array(n_img_test_array).reshape(-1,img_len,img_wid,1)

In [ ]:
#plt.imshow(img_test_array,'gray')
y_p=cnn.predict(test_pctr)
prediction_index=np.argmax(y_p)
car_names[prediction_index]